In [1]:
import libmoon
from libmoon.solver.gradient.methods import EPOSolver
from libmoon.util.synthetic import synthetic_init
from libmoon.util.prefs import get_uniform_pref
from libmoon.util import get_problem


In [ ]:

# problem = get_problem(problem_name='ZDT1')
# prefs = get_uniform_pref(n_prob=2, n_obj=problem.n_obj, clip_eps=1e-2)
# solver = EPOSolver(step_size=1e-2, n_epoch=1000, tol=1e-2, problem=problem, prefs=prefs)
# solver.verbose = True
# res = solver.solve(x_init=synthetic_init(problem, prefs))




In [ ]:

# import numpy as np
# import matplotlib.pyplot as plt

# from scipy.optimize import fsolve

# # Sensitivity functions
# def S_alpha_xss_analytic(xss: float, alpha: float, n: float) -> float:
#     numer = alpha * (1 + xss ** n)
#     denom = xss + alpha * n * xss ** n + 2 * xss ** (1 + n) + xss ** (1 + 2 * n)
#     sensitivity = numer / denom
#     return abs(sensitivity)

# def S_n_xss_analytic(xss: float, alpha: float, n: float) -> float:
#     numer = alpha * n * np.log(xss) * xss ** (n - 1)
#     denom = 1 + alpha * n * xss ** (n - 1) + 2 * xss ** n + xss ** (2 * n)
#     sensitivity = -numer / denom
#     return abs(sensitivity)

# # ODE and root finding helpers
# def Equ1(x: float, alpha: float, n: float) -> float:
#     return (alpha / (1 + x ** n)) - x

# def Equs(x: np.ndarray, t: float, params: np.ndarray) -> np.ndarray:
#     alpha, n = params
#     return np.array([Equ1(x[0], alpha, n)])

# def generate_initial_guesses(alpha_val: float, n_val: float):
#     # Provide multiple starting points to help fsolve converge
#     return [np.array([2.0]), np.array([0.5]), np.array([4.627])]

# def ssfinder(alpha_val: float, n_val: float) -> float:
#     params = np.array([alpha_val, n_val])
#     for guess in generate_initial_guesses(alpha_val, n_val):
#         xss, info, flag, _ = fsolve(
#             Equs, guess, args=(0.0, params), full_output=True, xtol=1e-8
#         )
#         if (flag == 1 and xss[0] > 0.04 and np.linalg.norm(info["fvec"]) < 1e-8):
#             # quick stability check
#             d = 1e-8
#             jac = (Equs(xss + d, 0.0, params) - Equs(xss, 0.0, params)) / d
#             if np.real(jac)[0] < 0:
#                 return xss[0]
#     return np.nan


In [6]:
# # Define parameter ranges
# alpha_bounds = (0.1, 8.0)
# n_bounds = (0.001, 5.0)

# num_alpha = 50  # number of samples along alpha
# num_n = 50      # number of samples along n

# alpha_vals = np.linspace(alpha_bounds[0], alpha_bounds[1], num_alpha)
# n_vals = np.linspace(n_bounds[0], n_bounds[1], num_n)

# # Collect all valid objective values
# samples = []

# for alpha in alpha_vals:
#     for n in n_vals:
#         xss = ssfinder(alpha, n)
#         if not np.isnan(xss) and xss > 0:
#             f1 = S_alpha_xss_analytic(xss, alpha, n)
#             f2 = S_n_xss_analytic(xss, alpha, n)
#             if not (np.isnan(f1) or np.isnan(f2) or np.isinf(f1) or np.isinf(f2)):
#                 samples.append((alpha, n, f1, f2))

# print(f"Number of valid samples: {len(samples)}")

# # Identify non-dominated (Pareto) points
# samples_sorted = sorted(samples, key=lambda x: x[2])  # sort by f1
# pareto_samples = []
# min_f2 = float('inf')
# for s in samples_sorted:
#     if s[3] <= min_f2:
#         pareto_samples.append(s)
#         min_f2 = s[3]

# print(f"Number of Pareto-optimal samples: {len(pareto_samples)}")

# # Plot the Pareto front
# f1_vals = [s[2] for s in pareto_samples]
# f2_vals = [s[3] for s in pareto_samples]

# plt.figure(figsize=(6, 4))
# plt.scatter(f1_vals, f2_vals, s=10, color='blue')
# plt.xlabel("Sensitivity to α (|d xss/dα|)")
# plt.ylabel("Sensitivity to n (|d xss/dn|)")
# plt.title("Pareto Front from Sampling")
# plt.grid(True)
# plt.tight_layout()
# plt.show()


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

torch.set_default_dtype(torch.float32)
torch.manual_seed(0)

# ---- smooth helpers -------------------------------------------------
def smooth_abs(z, eps=1e-8):
    return torch.sqrt(z * z + eps)

def equ1_torch(x, alpha, n, min_x=0.04):
    x     = torch.clamp(x,   min=min_x)
    alpha = torch.clamp(alpha, min=1e-6)
    n     = torch.clamp(n,     min=1e-6)
    return alpha / (1.0 + x.pow(n)) - x

def S_alpha_xss_torch(x, alpha, n, eps=1e-8):
    num = alpha * (1 + x.pow(n))
    den = x + alpha * n * x.pow(n) + 2 * x.pow(1 + n) + x.pow(1 + 2 * n)
    return smooth_abs(num / den, eps)

def S_n_xss_torch(x, alpha, n, eps=1e-8):
    x_safe = torch.clamp(x, min=1e-8)
    num = alpha * n * torch.log(x_safe) * x.pow(n - 1)
    den = 1 + alpha * n * x.pow(n - 1) + 2 * x.pow(n) + x.pow(2 * n)
    return smooth_abs(-num / den, eps)


In [ ]:
from libmoon.problem.synthetic.mop import BaseMOP

class SensitivityMOP(BaseMOP):
    """LibMOON-compatible problem for the two sensitivities."""
    def __init__(self, penalty_weight=200.0, eps=1e-8):
        super().__init__(n_var=3, n_obj=2, n_cons=0)
        # required by LibMOON helpers
        self.problem_name = "ZDT1"
        self.n_dim = self.n_var = 3
        self.n_obj = 2
        # bounds: alpha∈[0.1,8], n∈[0.1,5], x∈[0.04,10]
        self.lbound = torch.tensor([0.1, 0.1, 0.04])
        self.ubound = torch.tensor([8.0, 5.0, 10.0])
        self.penalty_w = float(penalty_weight)
        self.eps = float(eps)

    # torch evaluation – gradient friendly
    def _evaluate_torch(self, z: torch.Tensor) -> torch.Tensor:
        alpha, n, x = z[:, 0], z[:, 1], z[:, 2]
        res = equ1_torch(x, alpha, n, min_x=float(self.lbound[2]))
        pen = self.penalty_w * res.pow(2)
        f1 = S_alpha_xss_torch(x, alpha, n, eps=self.eps) + pen
        f2 = S_n_xss_torch(   x, alpha, n, eps=self.eps) + pen
        return torch.stack([f1, f2], dim=1)  # (B,2)


In [4]:
def rand_init(problem: BaseMOP, K: int, device=None):
    lb, ub = problem.lbound, problem.ubound
    if device:
        lb, ub = lb.to(device), ub.to(device)
    return lb + torch.rand(K, problem.n_dim, device=lb.device) * (ub - lb)


In [8]:
from libmoon.solver.gradient.methods import EPOSolver
from libmoon.util.prefs import get_uniform_pref

device = "cuda" if torch.cuda.is_available() else "cpu"

problem = SensitivityMOP(penalty_weight=200.0)
K = 40
prefs = get_uniform_pref(n_prob=K, n_obj=problem.n_obj, clip_eps=1e-2)

x0 = rand_init(problem, K, device=device)

solver = EPOSolver(
    step_size=5e-3,
    n_epoch=1500,
    tol=1e-4,
    problem=problem,
    prefs=prefs
)
res = solver.solve(x_init=x0)

X_raw = res.x.detach()
Y_raw = res.y.detach()


KeyError: 'CUSTOM'